In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# 기본 데이터 확인

In [2]:
df_2023 = pd.read_csv("./tmdb_v3_output/tmdb_2023.csv")
df_2023.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,3ão: An Endless Story,2023-12-11,movie,150.0,NaN,NaN,"Family, Adventure",Luís Felipe,"Victor Rodrigues, Thiago Gabriel, Ana Júlia, A...",BR,pt,VC Produções,This movie is a gift for the 2023 ESCA high sc...,3ão,8.375,4
1,My Fault,2023-06-08,movie,117.0,NaN,NaN,"Romance, Drama",Domingo González,"Nicole Wallace, Gabriel Guevara, Marta Hazas, ...",ES,es,"Pokeepsie Films, Amazon Studios","Noah must leave her city, boyfriend, and frien...","first time, forbidden love, stepbrother, femal...",7.800,3711
2,Brave Citizen,2023-10-25,movie,112.0,NaN,NaN,"Action, Drama, Comedy",Park Jin-pyo,"Shin Hye-sun, Lee Jun-young, Park Jung-woo, Pa...",KR,ko,"Studio N, Oscar 10 Studio, Vol Media","An expelled boxing champion, who now is a high...","secret identity, female fighter, female teache...",6.900,46
3,The Haunting at Saint Joseph's,2023-02-26,movie,100.0,NaN,NaN,"Horror, Thriller",Jon Cohen,"Tim Spriggs, Melissa Brattoni, Brandon Krum, D...",AU,en,NaN,"An engaged Muslim doctor, her fiancé, and thei...",NaN,4.500,12
4,Mobil Goyang,2023-04-24,movie,41.0,NaN,NaN,"Drama, Romance",Tomeng,"Virly Virginia, Uri Angelo, Radja Adipati",HN,ka,"Kelas Bintang, Karya Bintang Bersama",NaN,NaN,0.000,0


In [3]:
df_2023.sort_values(by='title', ascending=True).head(10)

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
7719,"""Moon rises in the Bay Area"" 2023 Greater Bay ...",2023-06-29,movie,212.0,NaN,NaN,Music,NaN,"Jackie Chan, Andy Lau Tak-Wah, Zhang Ziyi, Liu...",CN,zh,NaN,NaN,大湾区,6.500,4
4161,"""Truelove: The Film""",2023-09-12,movie,97.0,NaN,NaN,Documentary,Phil Viardo,"Callie Truelove, Christopher Knight, Carrie Un...",US,en,Former Prodigy Media,"""Truelove: The Film"" is a documentary feature ...","awareness, williams syndrome, peter brady, rar...",8.000,2
1187,#BOSSBABE,2023-08-08,movie,12.0,NaN,NaN,NaN,Kassy Gascho,"Katelyn Doyle, Pooja Bhandari, Daisy Lankstead...",CA,en,NaN,Set in a funhouse-mirror reflection of our wor...,NaN,8.333,3
18410,#Bat and Friends,2023-05-04,tv,NaN,1.0,1.0,NaN,NaN,NaN,US,en,NaN,Little Fox Level 01 Bat and Friends,NaN,0.000,0
11471,#BringBackAlice,2023-04-14,tv,53.0,1.0,6.0,"Mystery, Drama, Crime",NaN,"Helena Englert, Sebastian Dela, Bartłomiej Dek...",PL,pl,"Telemark, Głośno",After missing social media influencer Alicja S...,NaN,6.200,17
7708,#Cancelholtzman,2023-05-31,movie,55.0,NaN,NaN,Comedy,Mike Binder,Brian Holtzman,US,en,StandUpWorld Productions,A new stand-up special from comedy legend Bria...,"comedian, comedy club, stand-up",0.000,0
17248,#DEAD2ME,2023-09-24,tv,NaN,1.0,10.0,Documentary,NaN,NaN,GB,en,NaN,#DEAD2ME will explore the stories of ten Briti...,NaN,0.000,0
6781,#DoudouChallenge,2023-11-11,movie,6.0,NaN,NaN,"Animation, Family","Julie Majcher, Alexandra Delaunay-Fernandez, S...",NaN,FR,fr,Rubika,"On the road to her family vacation, Olivia, a ...","doudou, peluche, ardèche",0.000,0
2861,#MITO,2023-08-25,movie,78.0,NaN,NaN,"Thriller, Drama",Daisuke Miyazaki,"Tina Tamashiro, Hina Yukawa, Yu Inaba, Orakio,...",JP,ja,NaN,Yamane Mito lives her life as the charismatic ...,NaN,3.000,1
1231,#Manhole,2023-06-29,movie,99.0,NaN,NaN,"Drama, Thriller",Kazuyoshi Kumakiri,"Yuto Nakajima, Nao, Kento Nagayama, Haru Kuroki",JP,ja,"J Storm, GAGA Corporation",Kawamura is a promising young man with everyth...,NaN,6.300,20


In [4]:
df_2023.isnull().sum()

title                     0
release_date              0
type                      0
runtime                7876
season_count          10000
episode_count         10000
genre                  3042
director               7833
cast                   4081
country                  32
language                  0
production_company     7299
synopsis               4868
tags                  15753
imdb_rating               0
imdb_review_count         0
dtype: int64

# 2. 전처리 전처리_type

In [5]:
df_2023[df_2023['season_count'].isna() & df_2023['episode_count'].isna()].head(5)

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,3ão: An Endless Story,2023-12-11,movie,150.0,NaN,NaN,"Family, Adventure",Luís Felipe,"Victor Rodrigues, Thiago Gabriel, Ana Júlia, A...",BR,pt,VC Produções,This movie is a gift for the 2023 ESCA high sc...,3ão,8.375,4
1,My Fault,2023-06-08,movie,117.0,NaN,NaN,"Romance, Drama",Domingo González,"Nicole Wallace, Gabriel Guevara, Marta Hazas, ...",ES,es,"Pokeepsie Films, Amazon Studios","Noah must leave her city, boyfriend, and frien...","first time, forbidden love, stepbrother, femal...",7.800,3711
2,Brave Citizen,2023-10-25,movie,112.0,NaN,NaN,"Action, Drama, Comedy",Park Jin-pyo,"Shin Hye-sun, Lee Jun-young, Park Jung-woo, Pa...",KR,ko,"Studio N, Oscar 10 Studio, Vol Media","An expelled boxing champion, who now is a high...","secret identity, female fighter, female teache...",6.900,46
3,The Haunting at Saint Joseph's,2023-02-26,movie,100.0,NaN,NaN,"Horror, Thriller",Jon Cohen,"Tim Spriggs, Melissa Brattoni, Brandon Krum, D...",AU,en,NaN,"An engaged Muslim doctor, her fiancé, and thei...",NaN,4.500,12
4,Mobil Goyang,2023-04-24,movie,41.0,NaN,NaN,"Drama, Romance",Tomeng,"Virly Virginia, Uri Angelo, Radja Adipati",HN,ka,"Kelas Bintang, Karya Bintang Bersama",NaN,NaN,0.000,0


In [6]:
df_2023[df_2023['season_count'].isna() & df_2023['episode_count'].isna()].shape

(10000, 16)

In [7]:
df_2023['type'].value_counts(dropna=False)

type
movie    10000
tv       10000
Name: count, dtype: int64

In [8]:
print(df_2023[df_2023['type'] == 'movie']['season_count'].value_counts(dropna=False))
print(df_2023[df_2023['type'] == 'movie']['episode_count'].value_counts(dropna=False))

season_count
NaN    10000
Name: count, dtype: int64
episode_count
NaN    10000
Name: count, dtype: int64


In [9]:
print(df_2023[df_2023['type'] == 'tv']['season_count'].value_counts(dropna=False))

season_count
1.0     8496
2.0     1086
3.0      309
4.0       45
5.0       24
7.0        9
6.0        9
10.0       5
11.0       4
8.0        4
16.0       4
9.0        2
12.0       1
30.0       1
15.0       1
Name: count, dtype: int64


In [10]:
print(df_2023[df_2023['type'] == 'tv']['episode_count'].value_counts(dropna=False))

episode_count
1.0      1569
6.0       915
8.0       878
4.0       684
10.0      601
         ... 
219.0       1
225.0       1
280.0       1
278.0       1
166.0       1
Name: count, Length: 206, dtype: int64


In [11]:
df_2023_1 = df_2023

In [12]:
# movie인 경우 season/episode count가 NaN이면 1로 채움
df_2023_1.loc[df_2023_1['type'] == 'movie', ['season_count', 'episode_count']] = df_2023_1.loc[df_2023_1['type'] == 'movie', ['season_count', 'episode_count']].fillna(1)
pd.set_option('display.max_columns', None)
df_2023_1.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,3ão: An Endless Story,2023-12-11,movie,150.0,1.0,1.0,"Family, Adventure",Luís Felipe,"Victor Rodrigues, Thiago Gabriel, Ana Júlia, A...",BR,pt,VC Produções,This movie is a gift for the 2023 ESCA high sc...,3ão,8.375,4
1,My Fault,2023-06-08,movie,117.0,1.0,1.0,"Romance, Drama",Domingo González,"Nicole Wallace, Gabriel Guevara, Marta Hazas, ...",ES,es,"Pokeepsie Films, Amazon Studios","Noah must leave her city, boyfriend, and frien...","first time, forbidden love, stepbrother, femal...",7.800,3711
2,Brave Citizen,2023-10-25,movie,112.0,1.0,1.0,"Action, Drama, Comedy",Park Jin-pyo,"Shin Hye-sun, Lee Jun-young, Park Jung-woo, Pa...",KR,ko,"Studio N, Oscar 10 Studio, Vol Media","An expelled boxing champion, who now is a high...","secret identity, female fighter, female teache...",6.900,46
3,The Haunting at Saint Joseph's,2023-02-26,movie,100.0,1.0,1.0,"Horror, Thriller",Jon Cohen,"Tim Spriggs, Melissa Brattoni, Brandon Krum, D...",AU,en,NaN,"An engaged Muslim doctor, her fiancé, and thei...",NaN,4.500,12
4,Mobil Goyang,2023-04-24,movie,41.0,1.0,1.0,"Drama, Romance",Tomeng,"Virly Virginia, Uri Angelo, Radja Adipati",HN,ka,"Kelas Bintang, Karya Bintang Bersama",NaN,NaN,0.000,0


In [13]:
df_2023[(df_2023['type'] == 'tv') & (df_2023['season_count'].isna() | df_2023['episode_count'].isna())]
#df_2023[(df_2023['type'] == 'movie') & (df_2023['season_count'].isna() | df_2023['episode_count'].isna())]

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count


In [14]:
print(df_2023_1[df_2023_1['season_count'] == 1.0]['episode_count'].value_counts(dropna=False))

episode_count
1.0      11537
6.0        872
8.0        821
4.0        666
10.0       563
         ...  
115.0        1
151.0        1
55.0         1
117.0        1
64.0         1
Name: count, Length: 160, dtype: int64


In [15]:
print(df_2023_1[df_2023_1['episode_count'] == 1.0]['season_count'].value_counts(dropna=False))

season_count
1.0     11537
2.0        19
3.0         9
4.0         2
16.0        1
10.0        1
Name: count, dtype: int64


In [16]:
df_2023_2 = df_2023_1
df_2023_2.loc[df_2023_2['type'] == 'tv', ['season_count', 'episode_count']] = df_2023_2.loc[df_2023_1['type'] == 'tv', ['season_count', 'episode_count']].fillna(1)
pd.set_option('display.max_columns', None)
df_2023_2.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,3ão: An Endless Story,2023-12-11,movie,150.0,1.0,1.0,"Family, Adventure",Luís Felipe,"Victor Rodrigues, Thiago Gabriel, Ana Júlia, A...",BR,pt,VC Produções,This movie is a gift for the 2023 ESCA high sc...,3ão,8.375,4
1,My Fault,2023-06-08,movie,117.0,1.0,1.0,"Romance, Drama",Domingo González,"Nicole Wallace, Gabriel Guevara, Marta Hazas, ...",ES,es,"Pokeepsie Films, Amazon Studios","Noah must leave her city, boyfriend, and frien...","first time, forbidden love, stepbrother, femal...",7.800,3711
2,Brave Citizen,2023-10-25,movie,112.0,1.0,1.0,"Action, Drama, Comedy",Park Jin-pyo,"Shin Hye-sun, Lee Jun-young, Park Jung-woo, Pa...",KR,ko,"Studio N, Oscar 10 Studio, Vol Media","An expelled boxing champion, who now is a high...","secret identity, female fighter, female teache...",6.900,46
3,The Haunting at Saint Joseph's,2023-02-26,movie,100.0,1.0,1.0,"Horror, Thriller",Jon Cohen,"Tim Spriggs, Melissa Brattoni, Brandon Krum, D...",AU,en,NaN,"An engaged Muslim doctor, her fiancé, and thei...",NaN,4.500,12
4,Mobil Goyang,2023-04-24,movie,41.0,1.0,1.0,"Drama, Romance",Tomeng,"Virly Virginia, Uri Angelo, Radja Adipati",HN,ka,"Kelas Bintang, Karya Bintang Bersama",NaN,NaN,0.000,0


# 2. 데이터 전처리_runtime

In [17]:
df_2023_2.isnull().sum()

title                     0
release_date              0
type                      0
runtime                7876
season_count              0
episode_count             0
genre                  3042
director               7833
cast                   4081
country                  32
language                  0
production_company     7299
synopsis               4868
tags                  15753
imdb_rating               0
imdb_review_count         0
dtype: int64

In [18]:
df_2023_2.shape

(20000, 16)

In [19]:
df_2023_2[df_2023_2['runtime'].isna() & df_2023_2['genre'].isna() ].shape

(2412, 16)

## 2_2 runtime null 값 열 제거

In [20]:
df_2023_3 = df_2023_2

In [21]:
df_2023_3.dropna(subset=['runtime'], inplace=True)

In [22]:
df_2023_3.shape

(12124, 16)

# 2. 데이터 전처리_pr

In [23]:
df_2023_4 = df_2023_3
df_2023_4

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,3ão: An Endless Story,2023-12-11,movie,150.0,1.0,1.0,"Family, Adventure",Luís Felipe,"Victor Rodrigues, Thiago Gabriel, Ana Júlia, A...",BR,pt,VC Produções,This movie is a gift for the 2023 ESCA high sc...,3ão,8.375,4
1,My Fault,2023-06-08,movie,117.0,1.0,1.0,"Romance, Drama",Domingo González,"Nicole Wallace, Gabriel Guevara, Marta Hazas, ...",ES,es,"Pokeepsie Films, Amazon Studios","Noah must leave her city, boyfriend, and frien...","first time, forbidden love, stepbrother, femal...",7.800,3711
2,Brave Citizen,2023-10-25,movie,112.0,1.0,1.0,"Action, Drama, Comedy",Park Jin-pyo,"Shin Hye-sun, Lee Jun-young, Park Jung-woo, Pa...",KR,ko,"Studio N, Oscar 10 Studio, Vol Media","An expelled boxing champion, who now is a high...","secret identity, female fighter, female teache...",6.900,46
3,The Haunting at Saint Joseph's,2023-02-26,movie,100.0,1.0,1.0,"Horror, Thriller",Jon Cohen,"Tim Spriggs, Melissa Brattoni, Brandon Krum, D...",AU,en,NaN,"An engaged Muslim doctor, her fiancé, and thei...",NaN,4.500,12
4,Mobil Goyang,2023-04-24,movie,41.0,1.0,1.0,"Drama, Romance",Tomeng,"Virly Virginia, Uri Angelo, Radja Adipati",HN,ka,"Kelas Bintang, Karya Bintang Bersama",NaN,NaN,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19902,El cor de l’avi,2023-12-26,tv,48.0,1.0,2.0,NaN,NaN,NaN,ES,ca,El Terrat,NaN,NaN,0.000,0
19909,"My Mom, Your Dad",2023-08-08,tv,90.0,1.0,8.0,NaN,NaN,NaN,DE,de,NaN,NaN,NaN,0.000,0
19910,光照临川,2023-09-08,tv,25.0,1.0,1.0,Documentary,NaN,NaN,CN,zh,NaN,NaN,NaN,7.000,1
19917,EVNNE [Target: ME] SPECIAL,2023-12-26,tv,30.0,1.0,1.0,NaN,NaN,NaN,JP,ja,Space Shower TV,A super talented group consisting of Japanese ...,NaN,0.000,0


In [24]:
df_2023_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12124 entries, 0 to 19937
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               12124 non-null  object 
 1   release_date        12124 non-null  object 
 2   type                12124 non-null  object 
 3   runtime             12124 non-null  float64
 4   season_count        12124 non-null  float64
 5   episode_count       12124 non-null  float64
 6   genre               11494 non-null  object 
 7   director            9913 non-null   object 
 8   cast                11062 non-null  object 
 9   country             12110 non-null  object 
 10  language            12124 non-null  object 
 11  production_company  9035 non-null   object 
 12  synopsis            11132 non-null  object 
 13  tags                3952 non-null   object 
 14  imdb_rating         12124 non-null  float64
 15  imdb_review_count   12124 non-null  int64  
dtypes: float6

### tags 관련 기본 문제 해결

1) synopsis, tags, genre가 모두 NaN인 데이터 모두 제거

In [25]:
# 1. 세기: synopsis, genre가 모두 NaN인 행 개수
nan_rows = df_2023_4[['synopsis', 'genre']].isna().all(axis=1)
count_nan_rows = nan_rows.sum()
print(f"synopsis, genre가 모두 NaN인 행 개수: {count_nan_rows}")

synopsis, genre가 모두 NaN인 행 개수: 123


In [26]:
# 2. 삭제: 그런 행 삭제
df_2023_4_cleaned = df_2023_4.loc[~nan_rows].copy()

In [27]:
df_2023_4 = df_2023_4.loc[~nan_rows].copy()
df_2023_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12001 entries, 0 to 19937
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               12001 non-null  object 
 1   release_date        12001 non-null  object 
 2   type                12001 non-null  object 
 3   runtime             12001 non-null  float64
 4   season_count        12001 non-null  float64
 5   episode_count       12001 non-null  float64
 6   genre               11494 non-null  object 
 7   director            9883 non-null   object 
 8   cast                10983 non-null  object 
 9   country             11987 non-null  object 
 10  language            12001 non-null  object 
 11  production_company  8975 non-null   object 
 12  synopsis            11132 non-null  object 
 13  tags                3950 non-null   object 
 14  imdb_rating         12001 non-null  float64
 15  imdb_review_count   12001 non-null  int64  
dtypes: float6

In [28]:
df_2023_4

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,3ão: An Endless Story,2023-12-11,movie,150.0,1.0,1.0,"Family, Adventure",Luís Felipe,"Victor Rodrigues, Thiago Gabriel, Ana Júlia, A...",BR,pt,VC Produções,This movie is a gift for the 2023 ESCA high sc...,3ão,8.375,4
1,My Fault,2023-06-08,movie,117.0,1.0,1.0,"Romance, Drama",Domingo González,"Nicole Wallace, Gabriel Guevara, Marta Hazas, ...",ES,es,"Pokeepsie Films, Amazon Studios","Noah must leave her city, boyfriend, and frien...","first time, forbidden love, stepbrother, femal...",7.800,3711
2,Brave Citizen,2023-10-25,movie,112.0,1.0,1.0,"Action, Drama, Comedy",Park Jin-pyo,"Shin Hye-sun, Lee Jun-young, Park Jung-woo, Pa...",KR,ko,"Studio N, Oscar 10 Studio, Vol Media","An expelled boxing champion, who now is a high...","secret identity, female fighter, female teache...",6.900,46
3,The Haunting at Saint Joseph's,2023-02-26,movie,100.0,1.0,1.0,"Horror, Thriller",Jon Cohen,"Tim Spriggs, Melissa Brattoni, Brandon Krum, D...",AU,en,NaN,"An engaged Muslim doctor, her fiancé, and thei...",NaN,4.500,12
4,Mobil Goyang,2023-04-24,movie,41.0,1.0,1.0,"Drama, Romance",Tomeng,"Virly Virginia, Uri Angelo, Radja Adipati",HN,ka,"Kelas Bintang, Karya Bintang Bersama",NaN,NaN,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19833,少年剧有戏,2023-09-02,tv,25.0,1.0,1.0,Reality,NaN,NaN,CN,zh,NaN,NaN,NaN,0.000,0
19834,无敌王婿,2023-09-06,tv,1.0,1.0,2.0,Drama,NaN,NaN,CN,zh,NaN,NaN,NaN,0.000,0
19910,光照临川,2023-09-08,tv,25.0,1.0,1.0,Documentary,NaN,NaN,CN,zh,NaN,NaN,NaN,7.000,1
19917,EVNNE [Target: ME] SPECIAL,2023-12-26,tv,30.0,1.0,1.0,NaN,NaN,NaN,JP,ja,Space Shower TV,A super talented group consisting of Japanese ...,NaN,0.000,0


## director와 cast가 모두 nan인 행 제거

In [29]:
nan_both = df_2023_4[(df_2023_4['director'].isna()) & (df_2023_4['cast'].isna())]
print(f"director와 cast 둘 다 NaN인 행 개수: {len(nan_both)}")

director와 cast 둘 다 NaN인 행 개수: 429


In [30]:
df_2023_4 = df_2023_4.dropna(subset=['director', 'cast'], how='all')

In [31]:
df_2023_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11572 entries, 0 to 19937
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               11572 non-null  object 
 1   release_date        11572 non-null  object 
 2   type                11572 non-null  object 
 3   runtime             11572 non-null  float64
 4   season_count        11572 non-null  float64
 5   episode_count       11572 non-null  float64
 6   genre               11109 non-null  object 
 7   director            9883 non-null   object 
 8   cast                10983 non-null  object 
 9   country             11559 non-null  object 
 10  language            11572 non-null  object 
 11  production_company  8701 non-null   object 
 12  synopsis            10846 non-null  object 
 13  tags                3945 non-null   object 
 14  imdb_rating         11572 non-null  float64
 15  imdb_review_count   11572 non-null  int64  
dtypes: float6

# tag npl처리

In [32]:
# 1. tags 결측치는 빈 문자열로 대체
df_2023_4['tags'] = df_2023_4['tags'].fillna('')

# 2. synopsis, genre 결측치 처리 후 text_input 생성
df_2023_4['synopsis'] = df_2023_4['synopsis'].fillna('')
df_2023_4['genre'] = df_2023_4['genre'].fillna('')
df_2023_4['text_input'] = df_2023_4['synopsis'] + ' ' + df_2023_4['genre']

# 3. tags 기준으로 train / test 나누기
train_df = df_2023_4[df_2023_4['tags'] != ''].copy()
test_df = df_2023_4[df_2023_4['tags'] == ''].copy()

# 4. tags 리스트로 분리
train_df['tags_list'] = train_df['tags'].apply(lambda x: x.split(','))

# 5. 다중 라벨 인코딩
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_df['tags_list'])

# 6. 텍스트 벡터화 (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_df['text_input'])

# 7. 모델 학습 (LogisticRegression 다중 라벨)
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# 8. 예측 대상(test_df)이 비어있지 않으면 예측 수행
if not test_df.empty:
    X_test = vectorizer.transform(test_df['text_input'])
    y_prob = model.predict_proba(X_test)

    # 예측 임계값 설정 (낮춰서 태그 생성 늘림)
    threshold = 0.07
    y_pred = (y_prob >= threshold).astype(int)

    predicted_tags = mlb.inverse_transform(y_pred)
    test_df['predicted_tags'] = [', '.join(tags) if tags else np.nan for tags in predicted_tags]

    # 9. 규칙 기반 태그 보완용 키워드 룰 정의
    rules = {
        'romance': ['romance', 'love', 'relationship', 'affair', 'heart', 'kiss', 'wedding',
                    'boyfriend', 'girlfriend', 'breakup', 'dating', 'first love',
                    'valentine', 'romantic', 'jealousy', 'intimacy', 'honeymoon'],
        'action': ['battle', 'war', 'fight', 'soldier', 'gun', 'explosion', 'chase', 'enemy',
                'car chase', 'explosives', 'gunfight', 'hand-to-hand', 'sniper',
                'ambush', 'mercenary', 'adrenaline', 'combat', 'high-octane'],
        'thriller': ['thriller', 'mystery', 'suspense', 'crime', 'killer', 'murder', 'detective',
                    'interrogation', 'conspiracy', 'mind game', 'tension', 'abduction',
                    'dangerous', 'mind-bending', 'ticking clock', 'trap'],
        'comedy': ['comedy', 'funny', 'humor', 'laugh', 'joke', 'parody',
                'sitcom', 'sarcasm', 'one-liner', 'slapstick', 'banter',
                'awkward moment', 'gag', 'prank', 'hilarity'],
        'horror': ['ghost', 'horror', 'zombie', 'demon', 'vampire', 'curse',
                'haunted', 'scream', 'paranormal', 'supernatural', 'bloodcurdling',
                'creepy', 'terrifying', 'evil spirit', 'possessed', 'ritual'],
        'fantasy': ['magic', 'wizard', 'dragon', 'kingdom', 'mythical',
                    'sorcerer', 'enchanted', 'portal', 'spell', 'epic quest',
                    'mystical creature', 'ancient prophecy', 'grimoire'],
        'sci-fi': ['space', 'alien', 'robot', 'future', 'technology', 'sci-fi', 'cyber',
                'cyborg', 'AI uprising', 'time machine', 'spaceship', 'hyperspace',
                'galaxy', 'terraforming', 'futuristic', 'parallel universe', 'bioengineering'],
        'drama': ['family', 'life', 'struggle', 'emotional', 'drama', 'society',
                'inner struggle', 'self-discovery', 'character driven', 'emotional arc',
                'breakdown', 'life lesson', 'interpersonal', 'catharsis'],
        'crime': ['criminal', 'robbery', 'heist', 'mafia', 'gang',
                'heist', 'detective', 'undercover', 'criminal syndicate',
                'drug cartel', 'interrogation room', 'forensics', 'manhunt'],
        'animation': ['animation', 'cartoon', 'drawn', 'animated'],
        'sports': ['sports', 'football', 'basketball', 'coach', 'tournament'],
        'historical': ['king', 'queen', 'dynasty', 'empire', 'history', 'ancient'],
        'adventure': ['journey', 'quest', 'treasure', 'explore', 'island', 'survival'],
        'musical': ['song', 'singing', 'performance', 'dance', 'stage', 'music'],
        'documentary': ['real', 'true story', 'interview', 'archive', 'biography', 'footage'],
        'psychological': ['mind', 'mental', 'insanity', 'delusion', 'subconscious', 'dream'],
        'superhero': ['hero', 'superpower', 'villain', 'save', 'city', 'mask'],
        'noir': ['detective', 'dark', 'cigarette', 'femme fatale', 'shadow', 'monologue'],
        'family': ['parent', 'child', 'home', 'holiday', 'siblings', 'bond'],
        'rom-com': ['love', 'funny', 'awkward', 'date', 'charm', 'happy ending'],
        'military': ['army', 'navy', 'mission', 'rank', 'base', 'operation'],
        'cyberpunk': ['neon', 'hacker', 'corporation', 'dystopia', 'implant', 'surveillance'],
        'post-apocalyptic': ['ruins', 'survivor', 'radiation', 'collapse', 'tribe', 'scavenge'],
        'teen': ['high school', 'prom', 'friendship', 'coming of age', 'crush', 'rebellion'],
        'feel-good': ['inspiring', 'heartwarming', 'uplifting', 'positive', 'hope', 'smile'],
        'dark': ['depressing', 'grim', 'tragic', 'bleak', 'morbid'],
        'female-lead': ['strong woman', 'heroine', 'female protagonist', 'girl power'],
        'twist': ['plot twist', 'unpredictable', 'reveal', 'surprise ending'],
        'based-on-true-story': ['true story', 'biopic', 'real events', 'true events'],
        'underdog': ['struggle', 'rise', 'overcome', 'unlikely hero', 'against all odds'],
        'ensemble': ['many characters', 'multiple leads', 'group dynamic', 'interwoven stories'],
        'nostalgic': ['retro', 'classic', 'throwback', '80s', '90s', 'childhood'],
        'social-issue': ['racism', 'inequality', 'poverty', 'justice', 'activism'],
        'trending-theme': ['AI', 'climate', 'pandemic', 'influencer', 'metaverse'],
        'revenge': ['revenge', 'vengeance', 'payback', 'retaliation', 'score to settle'],
        'detective': ['detective', 'investigation', 'clues', 'case', 'inspector', 'sleuth'],
        'time-travel': ['time travel', 'time loop', 'past and future', 'timeline', 'chronosphere'],
        'courtroom': ['courtroom', 'trial', 'lawyer', 'judge', 'defendant', 'testimony'],
        'survival': ['survival', 'wilderness', 'isolation', 'extreme conditions', 'barely escape'],
        'espionage': ['spy', 'espionage', 'agent', 'undercover', 'intel', 'classified'],
        'campus': ['college', 'campus', 'lecture hall', 'seniors', 'dorm', 'professor'],
        'startup': ['startup', 'entrepreneur', 'founder', 'venture', 'pitching', 'angel investor'],
        'heist': ['heist', 'robbery', 'master plan', 'getaway', 'vault', 'crew'],
        'royalty': ['prince', 'princess', 'royal', 'crown', 'throne', 'palace'],
        'wuxia': ['martial arts', 'swordplay', 'wuxia', 'clan', 'sect', 'inner energy'],
        'idol': ['idol', 'stage', 'fandom', 'trainee', 'debut', 'fan meeting'],
        'political': ['election', 'politics', 'scandal', 'congress', 'corruption', 'regime'],
        'revenge-romance': ['bitter love', 'masked identity', 'burning passion', 'payback with love'],
        'healing-romance': ['gentle romance', 'emotional support', 'quiet relationship', 'warm affection'],
        'coming-of-age': ['self-discovery', 'growing up', 'teenage years', 'puberty',
                        'youth', 'maturity', 'identity', 'high school', 'first time', 'life lessons'],
        'dark-comedy': ['dark humor', 'macabre laugh', 'twisted joke', 'morbidly funny',
                        'ironic death', 'satirical tragedy', 'deadpan', 'black comedy'],
        'dystopian': ['oppressive regime', 'totalitarian', 'no freedom', 'surveillance state',
                    'rebellion', 'social collapse', 'harsh world', 'authoritarian', 'controlled society'],
        'slice-of-life': ['daily routine', 'ordinary moments', 'casual life', 'everyday drama',
                        'subtle emotion', 'mundane beauty', 'life as it is', 'quiet narrative'],
        'found-family': ['bond', 'unrelated family', 'loyal companions', 'chosen family',
                        'support system', 'teamwork', 'squad', 'surrogate parents', 'emotional ties'],
        'road-trip': ['on the road', 'journey across', 'miles', 'car travel', 'hit the road',
                    'cross-country', 'travel companions', 'gas station', 'stopover', 'long drive'],
        'tech-noir': ['grim future', 'tech-savvy criminal', 'neon-lit', 'cyber corruption',
                    'urban decay', 'AI rebellion', 'cyberspace', 'digital heist', 'surveillance overload'],
        'folklore': ['legend', 'traditional tale', 'mythical creature', 'ancient lore',
                    'cultural story', 'spirit', 'oral tradition', 'forest demon', 'village tale']
    }

    def simple_tag_rule(text):
        text = text.lower()
        tags = set()
        for tag, keywords in rules.items():
            if any(word in text for word in keywords):
                tags.add(tag)
        return ', '.join(tags) if tags else np.nan

    # 10. 예측 실패 (nan)인 경우 규칙 기반으로 태그 보완
    no_tag_idx = test_df[test_df['predicted_tags'].isna()].index
    test_df.loc[no_tag_idx, 'predicted_tags'] = test_df.loc[no_tag_idx, 'text_input'].apply(simple_tag_rule)

    # 11. 원본에 태그 삽입
    df_2023_4.loc[test_df.index, 'tags'] = test_df['predicted_tags']

C:\Users\User\AppData\Local\Temp\ipykernel_20476\1317382314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['tags'] = df_2023_4['tags'].fillna('')
C:\Users\User\AppData\Local\Temp\ipykernel_20476\1317382314.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['synopsis'] = df_2023_4['synopsis'].fillna('')
C:\Users\User\AppData\Local\Temp\ipykernel_20476\1317382314.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [33]:
df_2023_4['tags'].value_counts(dropna=False)

tags
 lgbt                                                                                                                                                                                                  229
drama                                                                                                                                                                                                  216
NaN                                                                                                                                                                                                    186
documentary                                                                                                                                                                                            154
 gay theme,  lgbt                                                                                                                                                                      

# country에서 nan은 language와 같은 값으로 처리

In [34]:
df_2023_4.isnull().sum()

title                    0
release_date             0
type                     0
runtime                  0
season_count             0
episode_count            0
genre                    0
director              1689
cast                   589
country                 13
language                 0
production_company    2871
synopsis                 0
tags                   186
imdb_rating              0
imdb_review_count        0
text_input               0
dtype: int64

In [35]:
df_2023_4['country'].fillna(df_2023_4['language'], inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_20476\2459764385.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2023_4['country'].fillna(df_2023_4['language'], inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_20476\2459764385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['country'].fillna(df_2023_4['language'], inpla

In [36]:
df_2023_4.isnull().sum()

title                    0
release_date             0
type                     0
runtime                  0
season_count             0
episode_count            0
genre                    0
director              1689
cast                   589
country                  0
language                 0
production_company    2871
synopsis                 0
tags                   186
imdb_rating              0
imdb_review_count        0
text_input               0
dtype: int64

In [37]:
df_2023_4['production_company'].value_counts(dropna=False)

production_company
NaN                                           2871
Vivamax                                         33
iQIYI                                           30
Tencent Video                                   22
Fuji Television Network                         19
                                              ... 
Tips Industries, 12th Street Entertainment       1
Dirigo Entertainment                             1
ENBU Seminar                                     1
Imani Media Group                                1
Mediawan Thematics                               1
Name: count, Length: 7459, dtype: int64

# production_company, director, cast _is_comming 처리
- 결측 값이 많아 이렇게 진행
- 정보 부족이지 하나의 회사명, 이름은 x
- 'unknown'만 삽입했을 경우 정상 값이라고 학습하는 것을 방지 => unknown으로 채우되 값은 없음으로 지정하기 위해 사용

In [38]:
unique_count = df_2023_4['production_company'].nunique(dropna=True)
print(unique_count)

7458


In [39]:
df_2023_4['production_company_is_missing'] = df_2023_4['production_company'].isna().astype(int)
df_2023_4['production_company'] = df_2023_4['production_company'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_20476\1728545354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['production_company_is_missing'] = df_2023_4['production_company'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_20476\1728545354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['production_company'] = df_2023_4['production_company'].fillna('unknown')


In [40]:
df_2023_4['cast_is_missing'] = df_2023_4['cast'].isna().astype(int)
df_2023_4['cast'] = df_2023_4['cast'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_20476\1612665721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['cast_is_missing'] = df_2023_4['cast'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_20476\1612665721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['cast'] = df_2023_4['cast'].fillna('unknown')


In [41]:
df_2023_4['director_is_missing'] = df_2023_4['director'].isna().astype(int)
df_2023_4['director'] = df_2023_4['director'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_20476\2335980419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['director_is_missing'] = df_2023_4['director'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_20476\2335980419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_4['director'] = df_2023_4['director'].fillna('unknown')


In [42]:
print(df_2023_4.shape)
print(df_2023_4.isnull().sum())

(11572, 20)
title                              0
release_date                       0
type                               0
runtime                            0
season_count                       0
episode_count                      0
genre                              0
director                           0
cast                               0
country                            0
language                           0
production_company                 0
synopsis                           0
tags                             186
imdb_rating                        0
imdb_review_count                  0
text_input                         0
production_company_is_missing      0
cast_is_missing                    0
director_is_missing                0
dtype: int64


In [43]:
nan_tags_rows = df_2023_4[df_2023_4['tags'].isna()]
nan_tags_rows

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,cast_is_missing,director_is_missing
223,Kill Shot,2023-08-15,movie,93.0,1.0,1.0,Action,Ari Novak,"Rib Hillis, Rachel Cook, Todd Gordon, Ari Nova...",US,en,"Pixium Film Group, Speed Merchant Productions,...","Posing as hunters, a group of terrorists are i...",NaN,9.1,161,"Posing as hunters, a group of terrorists are i...",0,0,0
707,太陽の戦士レオーナ SEASONⅡ 史上最悪の強敵襲来！[前編] -宇宙魔少女アローマ&教師...,2023-03-10,movie,75.0,1.0,1.0,,Sosuke Higashimura,"Narumi Okawa, Akane Kagurazaka, 田中勇紀",JP,ja,Zen Pictures,Straight to video,NaN,0.0,0,Straight to video,0,0,0
1112,The YouTube Effect,2023-07-07,movie,99.0,1.0,1.0,Documentary,Alex Winter,"Steve Chen, Jillian C. York, Hany Farid, Talia...",US,en,"XRM Media, Olive Hill Media, TIME Studios, Val...",YouTube has garnered over 2.3 billion users an...,NaN,4.3,5,YouTube has garnered over 2.3 billion users an...,0,0,0
1302,Tom Hanks: The Nomad,2023-05-20,movie,48.0,1.0,1.0,Documentary,Jake Hickman,"Josie Ellis, Barack Obama, Tom Hanks, Tim Alle...",GB,en,Entertain Me Productions,From humble beginnings to an aspiring thespian...,NaN,5.5,12,From humble beginnings to an aspiring thespian...,0,0,0
1895,League of Gods: The Fall of Sheng,2023-07-14,movie,80.0,1.0,1.0,Action,unknown,"Yao Di, Wang Qunshi, Qi Wu, Jing Gangshan, Kua...",CN,zh,unknown,,NaN,5.0,2,Action,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17966,Fake Show-Diffidate delle imitazioni,2023-09-18,tv,110.0,1.0,1.0,Talk,unknown,Max Giusti,IT,it,unknown,,NaN,0.0,0,Talk,1,0,1
18103,100% Poker,2023-09-24,tv,26.0,2.0,12.0,Talk,unknown,"Erika Moulet, Willy Dias, PonceP",FR,fr,"M6 Créations, M6",,NaN,0.0,0,Talk,0,0,1
18160,İki,2023-03-19,tv,15.0,1.0,7.0,Talk,unknown,"Alp Ersönmez, Çağrı Sertel",TR,tr,unknown,,NaN,0.0,0,Talk,1,0,1
18473,NEWS Oyama's stylish camp!,2023-01-21,tv,31.0,1.0,1.0,,unknown,Keiichiro Koyama,JP,ja,unknown,NEWS Koyama is an outdoor program that propose...,NaN,0.0,0,NEWS Koyama is an outdoor program that propose...,1,0,1


In [44]:
df_2023_5 = df_2023_4[df_2023_4['tags'].notna()].copy()
df_2023_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11386 entries, 0 to 19937
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          11386 non-null  object 
 1   release_date                   11386 non-null  object 
 2   type                           11386 non-null  object 
 3   runtime                        11386 non-null  float64
 4   season_count                   11386 non-null  float64
 5   episode_count                  11386 non-null  float64
 6   genre                          11386 non-null  object 
 7   director                       11386 non-null  object 
 8   cast                           11386 non-null  object 
 9   country                        11386 non-null  object 
 10  language                       11386 non-null  object 
 11  production_company             11386 non-null  object 
 12  synopsis                       11386 non-null  obje

In [46]:
df_2023_5.to_csv('전처리_완료(csv)/2023년도 전처리.csv', index=False)